# Caching policies

In [1]:
import random
import requests
import pandas as pd
import time

In [2]:
r = requests.get("https://ms.sites.cs.wisc.edu/cs544/data/wi-stations/stations.txt")
r.raise_for_status()
stations = r.text.strip().split("\n")
stations = random.sample(stations, k=10) # 10 random stations
workload = random.choices(stations, k=100, weights=[0.3, 0.2] + [0.5/8]*8) # repeats

In [3]:
stations

['US1WIPK0024',
 'US1WIWK0065',
 'US1WIBR0041',
 'US1WIWS0006',
 'USC00474523',
 'USW00014897',
 'US1WIBT0025',
 'US1WIBN0018',
 'US1WIWB0017',
 'USC00472413']

In [4]:
workload[:10]

['US1WIWK0065',
 'US1WIWB0017',
 'US1WIBT0025',
 'US1WIBR0041',
 'US1WIPK0024',
 'US1WIPK0024',
 'USC00472413',
 'US1WIWK0065',
 'US1WIPK0024',
 'US1WIBR0041']

In [6]:
station = 'USC00478267'
df = pd.read_csv(f"https://ms.sites.cs.wisc.edu/cs544/data/wi-stations/{station}.csv.gz",
                         names=["station", "date", "element", "value", "m", "q", "s", "obs"], low_memory=False)
df.head(3)

,station,date,element,value,m,q,s,obs
0,USC00478267,19050301,PRCP,0,NaN,NaN,0,NaN
1,USC00478267,19050302,PRCP,0,NaN,NaN,0,NaN
2,USC00478267,19050303,PRCP,0,T,NaN,0,NaN


## FIFO

In [ ]:
cache_size = 3
cache = {}           # key=station name, value=DataFrame for that station
evict_order = []     # start of list contains items to be evicted (end of list is freshest)
# TODO: use a faster data struct for evict_order than is not O(N) for pop(0)

# stats
hits = [] # True(hit), False(miss)
ms_latencies = []

def get_station(station):
    start = time.time()
    if station in cache:
        print("hit", end=", ")
        hits.append(True)
        df = cache[station]
    else:
        print("miss", end=", ")
        hits.append(False)
        df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs544/data/wi-stations/{station}.csv.gz",
                             names=["station", "date", "element", "value", "m", "q", "s", "obs"], low_memory=False)

        cache[station] = df
        evict_order.append(station)
        
        # should we evict?
        if len(cache) > cache_size:
            victim = evict_order.pop(0)  # pop from the front
            cache.pop(victim)

    end = time.time()
    ms = (end-start) * 1000
    ms_latencies.append(ms)

    return df

for station in workload:
    df = get_station(station)
    #print(station, evict_order)

In [ ]:
sum(hits) / len(hits)

In [ ]:
sum(ms_latencies) / len(ms_latencies)

## LRU

In [ ]:
cache_size = 3
cache = {}   # key=station name, value=DataFrame for that station
evict_order = []     # start of list contains items to be evicted (end of list is freshest)
# TODO: use a faster data struct for evict_order than is not O(N) for pop(0)

# stats
hits = [] # True(hit), False(miss)
ms_latencies = []

def get_station(station):
    start = time.time()
    if station in cache:
        print("hit", end=", ")
        hits.append(True)
        df = cache[station]

        evict_order.remove(station)
        evict_order.append(station)
    else:
        print("miss", end=", ")
        hits.append(False)
        df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs544/data/wi-stations/{station}.csv.gz",
                             names=["station", "date", "element", "value", "m", "q", "s", "obs"], low_memory=False)

        cache[station] = df
        evict_order.append(station)
        
        # should we evict?
        if len(cache) > cache_size:
            victim = evict_order.pop(0)  # pop from the front
            cache.pop(victim)

    end = time.time()
    ms = (end-start) * 1000
    ms_latencies.append(ms)

    return df

for station in workload:
    df = get_station(station)
    #print(station, evict_order)

In [ ]:
sum(hits) / len(hits)

In [ ]:
sum(ms_latencies) / len(ms_latencies)